In [74]:
from util._io import read_config
from collections import namedtuple
from pathlib import Path
import pandas as pd
import numpy as np
import fastsca
import logging
import _rpstruct as rp
import _pnmf
from sklearn.decomposition import NMF
from sklearn import preprocessing
import random
from scipy.special import softmax
from sklearn.metrics.cluster import contingency_matrix,silhouette_score,adjusted_rand_score,calinski_harabasz_score,davies_bouldin_score
np.random.seed(42)

experiment = '/projects/experiments/asapp/'
server = Path.home().as_posix()
experiment_home = server+experiment
experiment_config = read_config(experiment_home+'config.yaml')
args = namedtuple('Struct',experiment_config.keys())(*experiment_config.values())

sca = fastsca.FASTSCA()
sca.config = args
sca.initdata()
# sca.loaddata()
# print(sca.data.mtx.shape)
fn = sca.config.home + sca.config.experiment +sca.config.output + sca.config.sample_id+'/'+sca.config.sample_id



In [61]:
df_y = pd.read_csv(fn+'_rp_bulk.csv.gz')
df_y.shape


(1146, 19798)

In [11]:
import numpy as np
import pandas as pd
from numpy.random import normal
np.random.seed(37)

df_w = pd.DataFrame([
[0.8,0.15,0.05],
[0.75,0.15,0.1],
[0.8,0.15,0.05],
[0.75,0.15,0.1],
[0.15,0.05,0.8],
[0.15,0.05,0.8],
[0.15,0.05,0.8],
[0.15,0.05,0.8],
[0.05,0.8,0.15],
[0.05,0.8,0.15],
[0.05,0.8,0.15],
[0.05,0.8,0.15],
])

print('W m x k',df_w.shape)

a = list(normal(5,1,100)) + list(normal(1,1,100)) + list(normal(1,1,100))
b = list(normal(1,1,100)) + list(normal(1,1,100)) + list(normal(5,1,100))
c = list(normal(1,1,100)) + list(normal(5,1,100)) + list(normal(1,1,100))
df_h = pd.DataFrame([a,b,c])

print('H k x n',df_h.shape)


df_y = pd.DataFrame(np.dot(df_w.to_numpy(),df_h.to_numpy()))
print('A m x n',df_y.shape)


W m x k (12, 3)
H k x n (3, 300)
A m x n (12, 300)


In [80]:
import _pnmf,_dcpnmf
import importlib
importlib.reload(_dcpnmf)

# m = _pnmf.PoissonMF(n_components=3)
# res = m.fit(df_y.T.to_numpy())
m = _dcpnmf.DCPoissonMF(n_components=1)
res = m.fit(df_y.to_numpy())

/home/BCCRC.CA/ssubedi/projects/experiments/asapp/asapp/_dcpnmf.py:57: RuntimeWarning: invalid value encountered in double_scalars
  improvement = (bound - old_bd) / abs(old_bd)


	After ITERATION: 0	Objective: 3554065622.25	Old objective: -inf	Improvement: nan
3143345.6733879633
	After ITERATION: 1	Objective: 3554065622.25	Old objective: 3554065622.25	Improvement: -0.00000
3143345.6733918814




In [81]:
import _topics_util
import seaborn as sns

df_w = pd.DataFrame(res.EF)
df_w.columns = sca.data.cols
df_top = _topics_util.get_topic_top_genes(df_w,top_n=25)
# df_top = df_top.pivot(index='Topic',columns='Gene',values='Proportion')
# sns.clustermap(df_top.T,cmap='viridis')

/home/BCCRC.CA/ssubedi/projects/experiments/asapp/asapp/_topics_util.py:8: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only.
  gtab = df.T.iloc[:,x].sort_values(0,ascending=False)[:top_n].reset_index()


In [82]:
df_top

,Topic,GeneType,Genes,Gene,Proportion
0,k0,top_genes,g1,MALAT1,675.613949
1,k0,top_genes,g1,FTH1,192.048000
2,k0,top_genes,g1,EEF1A1,185.120208
3,k0,top_genes,g1,B2M,178.845106
4,k0,top_genes,g1,RPLP1,166.750023
5,k0,top_genes,g1,FTL,151.062793
6,k0,top_genes,g1,RPL10,140.140595
7,k0,top_genes,g1,TMSB4X,123.605606
8,k0,top_genes,g1,RPS12,122.442013
9,k0,top_genes,g1,ACTB,115.544804
